In [12]:
library(dplyr)
library(mice)
library(randomForest)
library(ineq)
library(data.table)
library(MLmetrics)
library(lightgbm)

In [31]:
train <- read.csv("train_ajEneEa.csv")

In [ ]:
train

In [33]:
str(train)

'data.frame':	43400 obs. of  12 variables:
 $ id               : int  30669 30468 16523 56543 46136 32257 52800 41413 15266 28674 ...
 $ gender           : Factor w/ 3 levels "Female","Male",..: 2 2 1 1 2 1 1 1 1 1 ...
 $ age              : num  3 58 8 70 14 47 52 75 32 74 ...
 $ hypertension     : int  0 1 0 0 0 0 0 0 0 1 ...
 $ heart_disease    : int  0 0 0 0 0 0 0 1 0 0 ...
 $ ever_married     : Factor w/ 2 levels "No","Yes": 1 2 1 2 1 2 2 2 2 2 ...
 $ work_type        : Factor w/ 5 levels "children","Govt_job",..: 1 4 4 4 3 4 4 5 4 5 ...
 $ Residence_type   : Factor w/ 2 levels "Rural","Urban": 1 2 2 1 1 2 2 1 1 2 ...
 $ avg_glucose_level: num  95.1 88 110.9 69 161.3 ...
 $ bmi              : num  18 39.2 17.6 35.9 19.1 50.1 17.7 27 32.3 54.6 ...
 $ smoking_status   : Factor w/ 4 levels "","formerly smoked",..: 1 3 1 2 1 1 2 3 4 3 ...
 $ stroke           : int  0 0 0 0 0 0 0 0 0 0 ...


In [34]:
summary(train[train$Residence_type == 'Rural',])

       id           gender           age         hypertension    
 Min.   :    4   Female:12816   Min.   : 0.08   Min.   :0.00000  
 1st Qu.:17999   Male  : 8822   1st Qu.:24.00   1st Qu.:0.00000  
 Median :36544   Other :    6   Median :44.00   Median :0.00000  
 Mean   :36324                  Mean   :42.19   Mean   :0.09448  
 3rd Qu.:54417                  3rd Qu.:60.00   3rd Qu.:0.00000  
 Max.   :72938                  Max.   :82.00   Max.   :1.00000  
                                                                 
 heart_disease    ever_married         work_type     Residence_type
 Min.   :0.0000   No : 7757    children     : 3082   Rural:21644   
 1st Qu.:0.0000   Yes:13887    Govt_job     : 2714   Urban:    0   
 Median :0.0000                Never_worked :   84                 
 Mean   :0.0481                Private      :12320                 
 3rd Qu.:0.0000                Self-employed: 3444                 
 Max.   :1.0000                                                 

In [35]:
summary(train[train$Residence_type == 'Urban',])

       id           gender           age         hypertension    
 Min.   :    1   Female:12849   Min.   : 0.08   Min.   :0.00000  
 1st Qu.:18055   Male  : 8902   1st Qu.:24.00   1st Qu.:0.00000  
 Median :36129   Other :    5   Median :44.00   Median :0.00000  
 Mean   :36329                  Mean   :42.25   Mean   :0.09266  
 3rd Qu.:54615                  3rd Qu.:60.00   3rd Qu.:0.00000  
 Max.   :72943                  Max.   :82.00   Max.   :1.00000  
                                                                 
 heart_disease     ever_married         work_type     Residence_type
 Min.   :0.00000   No : 7705    children     : 3074   Rural:    0   
 1st Qu.:0.00000   Yes:14051    Govt_job     : 2726   Urban:21756   
 Median :0.00000                Never_worked :   93                 
 Mean   :0.04693                Private      :12514                 
 3rd Qu.:0.00000                Self-employed: 3349                 
 Max.   :1.00000                                          

In [36]:
library(dplyr)

In [37]:
trainWoNULL <- NULL
trainWoNULL <- train %>% replace(. =="", NA)

In [38]:
summary(trainWoNULL)

       id           gender           age         hypertension    
 Min.   :    1   Female:25665   Min.   : 0.08   Min.   :0.00000  
 1st Qu.:18039   Male  :17724   1st Qu.:24.00   1st Qu.:0.00000  
 Median :36352   Other :   11   Median :44.00   Median :0.00000  
 Mean   :36326                  Mean   :42.22   Mean   :0.09357  
 3rd Qu.:54514                  3rd Qu.:60.00   3rd Qu.:0.00000  
 Max.   :72943                  Max.   :82.00   Max.   :1.00000  
                                                                 
 heart_disease     ever_married         work_type     Residence_type
 Min.   :0.00000   No :15462    children     : 6156   Rural:21644   
 1st Qu.:0.00000   Yes:27938    Govt_job     : 5440   Urban:21756   
 Median :0.00000                Never_worked :  177                 
 Mean   :0.04751                Private      :24834                 
 3rd Qu.:0.00000                Self-employed: 6793                 
 Max.   :1.00000                                          

In [39]:
write.csv(trainWoNULL,"trainWoNULL.csv")

In [40]:
traindf <- read.csv("trainWoNULL.csv")

In [41]:
summary(traindf)

       X               id           gender           age       
 Min.   :    1   Min.   :    1   Female:25665   Min.   : 0.08  
 1st Qu.:10851   1st Qu.:18039   Male  :17724   1st Qu.:24.00  
 Median :21701   Median :36352   Other :   11   Median :44.00  
 Mean   :21701   Mean   :36326                  Mean   :42.22  
 3rd Qu.:32550   3rd Qu.:54514                  3rd Qu.:60.00  
 Max.   :43400   Max.   :72943                  Max.   :82.00  
                                                               
  hypertension     heart_disease     ever_married         work_type    
 Min.   :0.00000   Min.   :0.00000   No :15462    children     : 6156  
 1st Qu.:0.00000   1st Qu.:0.00000   Yes:27938    Govt_job     : 5440  
 Median :0.00000   Median :0.00000                Never_worked :  177  
 Mean   :0.09357   Mean   :0.04751                Private      :24834  
 3rd Qu.:0.00000   3rd Qu.:0.00000                Self-employed: 6793  
 Max.   :1.00000   Max.   :1.00000                      

In [42]:
library(mice)

In [43]:
miceMod <- mice(traindf[, !names(traindf) %in% "stroke"], method="rf")  # perform mice imputation, based on random forests.


 iter imp variable
  1   1  bmi  smoking_status

In [44]:
miceOutput <- complete(miceMod)  # generate the completed data.

ERROR: Error in complete(miceMod): object 'miceMod' not found


In [ ]:
anyNA(miceOutput)

In [ ]:
View(miceOutput)

In [ ]:
miceOutput1 <- miceOutput[c(2,11,12)]

In [ ]:
traindfnew <- traindf[,-c(11,12)]
View(traindfnew)

In [ ]:
traindfFinal <- merge(traindfnew,miceOutput1,by="id")
View(traindfFinal)